In [1]:
import requests
from copy import deepcopy

use_real_puzzle_data = False # else use demo data
separator = '\n' if True else ','

github_site_user = "https://raw.githubusercontent.com/stephenhumphrey/"
repository_branch = "advent-of-code/main/2021/12/18/"

names_files = [("magnitudes","demo-magnitudes"),
               ("demo","demo-summations"),
               ("puzzle","puzzle-data")]

data = {source:{"name":source,
                "url":f"{github_site_user}{repository_branch}{file}.txt"} 
        for source,file in names_files}

for name,source in data.items():
    print(f"{name}: {source['url']}")

magnitudes: https://raw.githubusercontent.com/stephenhumphrey/advent-of-code/main/2021/12/18/demo-magnitudes.txt
demo: https://raw.githubusercontent.com/stephenhumphrey/advent-of-code/main/2021/12/18/demo-summations.txt
puzzle: https://raw.githubusercontent.com/stephenhumphrey/advent-of-code/main/2021/12/18/puzzle-data.txt


In [2]:
# cheap and dirty constants
infinity = 99999999999999999999999999

# allow for easily turning on and off local logging inside this notebook
do_verbose = False # set to true for copious trace logging
def throwaway_verbose(unused):
    return
verbose = print if do_verbose else throwaway_verbose

# for sanity, let's limit Jupyter from getting a firehose of logging,
# which it handles poorly
global_endless_loop = 999999999 # halts certain functions which might fail

In [3]:
# utility function for pretty-printing large data arrays
def verbose_list(plist_string,max_elements = 5):
    plist = eval(plist_string)
    if len(plist) <= max_elements:
        sub = f"{plist}"
    elif max_elements < 2:
        sub = f"[{plist[0]}, ...]"
    else:
        half_max = max_elements // 2
        sub = "["
        for v in plist[0:half_max]:
            sub += f"{v}, "
        sub += "... "
        for v in plist[-half_max:-1]:
            sub += f"{v}, "
        sub += f"{plist[-1]}]"
        
    print(f"len({plist_string})={len(plist)}, {plist_string}={sub}")

In [4]:
# download and roughly process our testing and production data sources
for source_name,source in data.items():
    raw_data = requests.get(source["url"]).text
        
    data_lines = [line for line in raw_data.split(separator) if len(line.strip()) > 0]
    source["raw_lines"] = data_lines
    
    if False:
        print(source_name)
        for line in data_lines:
            print(line)
    
    if source_name == "puzzle":
        source["operands_raw"] = [data_lines]
        source["expected_values"] = ["unknown"]
    else:
        right = []
        source["operands_raw"] = []
        for line in data_lines:
            line_data = [v.strip() for v in line.split('=') if len(v.strip())>1]
            left = [v.strip() for v in line_data[0].split('+')]
            source["operands_raw"].append(left)
            right.append(line_data[1])
        source["expected_values"] = right

In [5]:
# utility functions for dealing with this particular puzzle's data types
def separate_pair(pair_string):
    assert(pair_string[0]=='[' and pair_string[-1]==']')
    
    depth = 0
    split_at = 0
    for i,c in enumerate(pair_string):
        if c == '[':
            depth += 1
        elif c == ']':
            depth -= 1
        elif c == ',' and depth == 1:
            split_at = i
            break
    return pair_string[1:split_at], pair_string[split_at + 1:-1]

def parse_pair(pair_string):
    
    if pair_string[0] != '[':
        return int(pair_string)
    
    assert(pair_string[0]=='[' and pair_string[-1]==']')

    left,right = separate_pair(pair_string)
    left_parsed = parse_pair(left)
    right_parsed = parse_pair(right)
    
    return [left_parsed,right_parsed]

def pair_magnitude(pair):
    assert(isinstance(pair,list) and len(pair)==2)
    
    left,right = pair[0], pair[1]
    assert(isinstance(left,int) or isinstance(left,list))
    assert(isinstance(right,int) or isinstance(right,list))
    
    sum_left_right = 0
    if isinstance(left,int):
        sum_left_right += left * 3
    else:
        sum_left_right += pair_magnitude(left) * 3
    
    if isinstance(right,int):
        sum_left_right += right * 2
    else:
        sum_left_right += pair_magnitude(right) * 2

    return sum_left_right

In [6]:
# process and normalize our testing and production datasets so they share formats
for source_name, source in data.items():
    expected_values = source["expected_values"]
    pairs = []
    source["operands"] = pairs
    for i,operands in enumerate(source["operands_raw"]):
        
        verbose(f"{source_name}{i} operands({len(operands)}) expect: {expected_values[i]}")
        
        operand_pairs = []
        for j,operand in enumerate(operands):
            pair = parse_pair( operand )
            operand_pairs.append(pair)
            magnitude = pair_magnitude( pair )
            
            verbose(f"  operand[{j}] = {operand} pair = {pair} magnitude = {magnitude}")
        
        verbose(f"  pairs = {operand_pairs}")
        pairs.append(operand_pairs)

In [7]:
# this puzzle's pair summation, not counting "reduction" (as the puzzle defines it)
def unreduced_sum_of_two_pairs(pair0, pair1):
    if pair0 is None:
        return pair1
    if pair1 is None:
        return pair0
    return [pair0, pair1]

In [8]:
part_1_notes = """

To reduce a snailfish number, you must repeatedly do the first action in this list that applies to the snailfish number:

If any pair is nested inside four pairs, the leftmost such pair explodes.
If any regular number is 10 or greater, the leftmost such regular number splits.
Once no action in the above list applies, the snailfish number is reduced.

During reduction, at most one action applies, after which the process returns to the top of the list of actions. For example, if split produces a pair that meets the explode criteria, that pair explodes before other splits occur.

To explode a pair, the pair's left value is added to the first regular number to the left of the exploding pair (if any), and the pair's right value is added to the first regular number to the right of the exploding pair (if any). Exploding pairs will always consist of two regular numbers. Then, the entire exploding pair is replaced with the regular number 0.
Here are some examples of a single explode action:

[[[[[9,8],1],2],3],4] becomes [[[[0,9],2],3],4] (the 9 has no regular number to its left, so it is not added to any regular number).
[7,[6,[5,[4,[3,2]]]]] becomes [7,[6,[5,[7,0]]]] (the 2 has no regular number to its right, and so it is not added to any regular number).
[[6,[5,[4,[3,2]]]],1] becomes [[6,[5,[7,0]]],3].
[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]] becomes [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]] (the pair [3,2] is unaffected because the pair [7,3] is further to the left; [3,2] would explode on the next action).
[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]] becomes [[3,[2,[8,0]]],[9,[5,[7,0]]]].

To split a regular number, replace it with a pair; the left element of the pair should be the regular number divided by two and rounded down, while the right element of the pair should be the regular number divided by two and rounded up. For example, 10 becomes [5,5], 11 becomes [5,6], 12 becomes [6,6], and so on.

Here is the process of finding the reduced result of [[[[4,3],4],4],[7,[[8,4],9]]] + [1,1]:

after addition: [[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]
after explode:  [[[[0,7],4],[7,[[8,4],9]]],[1,1]]
after explode:  [[[[0,7],4],[15,[0,13]]],[1,1]]
after split:    [[[[0,7],4],[[7,8],[0,13]]],[1,1]]
after split:    [[[[0,7],4],[[7,8],[0,[6,7]]]],[1,1]]
after explode:  [[[[0,7],4],[[7,8],[6,0]]],[8,1]]

Once no reduce actions apply, the snailfish number that remains is the actual result of the addition operation: [[[[0,7],4],[[7,8],[6,0]]],[8,1]].

"""

In [9]:
# process summation of multiple pairs in an equation, sequentionally
def reduced_sum_of_pairs(pairs):
    assert(len(pairs) > 0)
    
    print(f"  called reduced_sum_of_pairs() with {pairs}")
    
    previous_sum = None
    for pair in pairs:
        verbose(f"  +++ adding {pair} to any previous result")
        
        unreduced_sum = unreduced_sum_of_two_pairs(previous_sum,pair)
        verbose(f"  *** unreduced_sum = {unreduced_sum}")
        
        while reduce( unreduced_sum ):
            verbose(f"  ??? partially_reduced = {unreduced_sum}")
            pass
        
        verbose(f"  === fully reduced = {unreduced_sum}")
        previous_sum = unreduced_sum        
    
    return previous_sum

In [10]:
# search the tree for a particular pair, finding the closest "regular" neighbor as well
def find_side_of(exploding_pair,parents,use_strongside = True):
    
    verbose(f"    calling find_side_of({'strong' if use_strongside else 'weak'}) "
          f"pair = {exploding_pair} parents = {parents}")
    
    found = False
    neighbor = None
    closer_neighbor = None
    immediate_parent = None
    
    if isinstance(parents,int): 
        # this level isn't a pair
        return found, neighbor, immediate_parent 
    
    strong_indices = (0,1) if use_strongside else (1,0)
    strong_index, weak_index = strong_indices
    
    strongside, weakside = parents[strong_index], parents[weak_index]
    
    if exploding_pair is strongside:
        # we are done searching; there can't be any closer strongside neighbor
        found = True
        immediate_parent = parents
        
    elif exploding_pair is weakside:
        # we found the pair, but there might still be a closer strongside neighbor
        found = True
        immediate_parent = parents
        
        # search only for a closer neighbor in the strongside tree
        if isinstance(strongside,int):
            neighbor = (strongside, parents, use_strongside)
        else:
            expectFalse, closer_neighbor, _ = find_side_of( exploding_pair,
                                                 strongside, use_strongside )
            assert(not expectFalse) # we better not find it again
        
    else:
        # we still haven't found the pair, it could be on either side
        # search the strongside tree first, and failing that, the weakside tree
        if isinstance(strongside,int):
            neighbor = (strongside, parents, use_strongside)
        else:
            found, closer_neighbor, immediate_parent = find_side_of( exploding_pair,
                                                        strongside, use_strongside )
        if closer_neighbor is not None:
            neighbor = closer_neighbor
            closer_neighbor = None
        
        if not found: 
            # the exploding pair wasn't anywhere in the strongside tree
            # search the weakside
            if isinstance(weakside,int):
                neighbor = (weakside, parents, not use_strongside)
            else:
                found, closer_neighbor, immediate_parent = find_side_of( exploding_pair,
                                                        weakside, use_strongside )
    
    # final cleanup, we may have found a closer neighbor than our prior closest
    if closer_neighbor is not None:
        neighbor = closer_neighbor
    
    verbose(f"    exiting find_side_of({'strong' if use_strongside else 'weak'}) "
          f"found = {'found' if found else 'not-found'}"
          f"neighbor = {neighbor} immediate_parent = {immediate_parent}")
    
    return found, neighbor, immediate_parent

In [11]:
# explode one pair in the tree, as defined by this particular puzzle
def explode(parents, exploding_pair):
    
    verbose(f"  called explode() with parents = {parents} exploding_pair = {exploding_pair}")
    
    left,right = exploding_pair[0], exploding_pair[1]
    
    leftside = find_side_of(exploding_pair,parents,use_strongside = True)
    rightside = find_side_of(exploding_pair,parents,use_strongside = False)
        
    verbose(f"    exploding with leftside = {leftside}")
    verbose(f"    exploding with rightside = {rightside}")
    
    _, neighbor, parent_of_exploding_pair = leftside
    
    if exploding_pair is parent_of_exploding_pair[0]:
        parent_of_exploding_pair[0] = 0
    else:
        parent_of_exploding_pair[1] = 0
    
    if neighbor is not None:
        value, neighbor_parent, on_left = neighbor
        neighbor_index = 0 if on_left else 1
        assert( value == neighbor_parent[ neighbor_index ] )
        neighbor_parent[ neighbor_index ] += left
    
    _, neighbor, _ = rightside
    
    if neighbor is not None:
        value, neighbor_parent, on_left = neighbor
        neighbor_index = 0 if on_left else 1
        assert( value == neighbor_parent[ neighbor_index ] )
        neighbor_parent[ neighbor_index ] += right
        
    return

In [12]:
# split one value in the tree, as defined by this particular puzzle
def split(parents, pair, splitting_value, on_left):
    
    verbose(f"  called split() with parents = {parents} pair = {pair} splitting_value = {splitting_value}")

    left = splitting_value // 2
    right = splitting_value - left
    new_pair = [left,right]
    
    if on_left:
        pair[0] = new_pair
    else:
        pair[1] = new_pair
    
    verbose(f"  exiting split() with parents = {parents} pair = {pair}")
        
    return

In [13]:
def split_first(parent,pair):
    
    verbose(f"  called split_first() with parent = {parent} pair = {pair}")
    
    if isinstance(pair,int):
        return False
    
    left,right = pair[0],pair[1]
    reduced = False
    
    if isinstance(left,int):
        if left >= 10:
            # split left!
            split(parent, pair, left, True)
            reduced = True
    else:
        reduced = split_first(pair,left)
        
    if not reduced:
        if isinstance(right,int):
            if right >= 10:
                split(parent, pair, right, False)
                reduced = True
        else:
            reduced = split_first(pair,right)
    
    return reduced

In [14]:
# make one reduction pass through the tree, caller will repeat until no reductions remain
def reduce(pair, parent = None, depth = 0):
    
    verbose(f"  called reduce() with {pair}, depth = {depth}")
    
    global global_endless_loop
    global_endless_loop -= 1
    assert( global_endless_loop > 0 )
    
    left,right = pair[0],pair[1]
    reduced = False
    
    if parent is None:
        parent = pair
    
    while True:
        pair[0],pair[1] = left,right
    
        if isinstance(left,int) and isinstance(right,int):
            # reg number
            if depth >= 4:
                # explode!
                explode(parent,pair)
                reduced = True
            
        else:
            # inner pairs or mixed regular-and-pair
            if not isinstance(left,int):
                if reduce(left, parent, depth + 1):
                    reduced = True
                    
            if not reduced and not isinstance(right,int):
                if reduce(right, parent, depth + 1):
                    reduced = True
        
        if not reduced and depth == 0:
            # ready for possible splits
            left, right = parent[0], parent[1]
            reduced = split_first(parent,left)
            
            if not reduced:
                reduced = split_first(parent,right)
        
        break # old: we didn't reduce on this pass through the loop
            # new: we probably don't need this loop at all anymore, with no continue
    
    return reduced

# some test reductions from the original puzzle description
test_reduce = {} # test-input --> expected-result
test_reduce["[[[[0,9],2],3],4]"] = [[[[[9,8],1],2],3],4]
test_reduce["[7,[6,[5,[7,0]]]]"] = [7,[6,[5,[4,[3,2]]]]]
test_reduce["[[6,[5,[7,0]]],3]"] = [[6,[5,[4,[3,2]]]],1]
test_reduce["[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]"] = [[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]
test_reduce["[[3,[2,[8,0]]],[9,[5,[7,0]]]]"] = [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]

for test_expected,test_pair in test_reduce.items():
    print(f"{60 * '='}\nTesting reduce( {test_pair} )")
    reduce(test_pair)
    print(f" One-pass reduce() = {test_pair}")
    print(f"Expecting reduce() = {test_expected.replace(',',', ')}")
    print(f"Magnitude reduce() = {pair_magnitude(test_pair)}")

Testing reduce( [[[[[9, 8], 1], 2], 3], 4] )
 One-pass reduce() = [[[[0, 9], 2], 3], 4]
Expecting reduce() = [[[[0, 9], 2], 3], 4]
Magnitude reduce() = 548
Testing reduce( [7, [6, [5, [4, [3, 2]]]]] )
 One-pass reduce() = [7, [6, [5, [7, 0]]]]
Expecting reduce() = [7, [6, [5, [7, 0]]]]
Magnitude reduce() = 285
Testing reduce( [[6, [5, [4, [3, 2]]]], 1] )
 One-pass reduce() = [[6, [5, [7, 0]]], 3]
Expecting reduce() = [[6, [5, [7, 0]]], 3]
Magnitude reduce() = 402
Testing reduce( [[3, [2, [1, [7, 3]]]], [6, [5, [4, [3, 2]]]]] )
 One-pass reduce() = [[3, [2, [8, 0]]], [9, [5, [4, [3, 2]]]]]
Expecting reduce() = [[3, [2, [8, 0]]], [9, [5, [4, [3, 2]]]]]
Magnitude reduce() = 769
Testing reduce( [[3, [2, [8, 0]]], [9, [5, [4, [3, 2]]]]] )
 One-pass reduce() = [[3, [2, [8, 0]]], [9, [5, [7, 0]]]]
Expecting reduce() = [[3, [2, [8, 0]]], [9, [5, [7, 0]]]]
Magnitude reduce() = 633


In [15]:
# assert(False) # stop here for now

In [16]:
for source_name in data.keys():
    source = data[source_name]
    expected_values = source["expected_values"]
    for i,saved_operands in enumerate(source["operands"]):
        
        operands = deepcopy(saved_operands)
        
        print(f"{60 * '='}\n{source_name}{i} operands({len(operands)}) ")      
        reduced_sum = reduced_sum_of_pairs(operands)
        print(f"  reduced_sum_of_pairs = {reduced_sum}")
        print(f"       expected result = {expected_values[i].replace(',',', ')}")  
        print(f"      magnitude of sum = {pair_magnitude(reduced_sum)}")

magnitudes0 operands(1) 
  called reduced_sum_of_pairs() with [[9, 1]]
  reduced_sum_of_pairs = [9, 1]
       expected result = 29
      magnitude of sum = 29
magnitudes1 operands(1) 
  called reduced_sum_of_pairs() with [[1, 9]]
  reduced_sum_of_pairs = [1, 9]
       expected result = 21
      magnitude of sum = 21
magnitudes2 operands(1) 
  called reduced_sum_of_pairs() with [[[9, 1], [1, 9]]]
  reduced_sum_of_pairs = [[9, 1], [1, 9]]
       expected result = 129
      magnitude of sum = 129
magnitudes3 operands(1) 
  called reduced_sum_of_pairs() with [[[1, 2], [[3, 4], 5]]]
  reduced_sum_of_pairs = [[1, 2], [[3, 4], 5]]
       expected result = 143
      magnitude of sum = 143
magnitudes4 operands(1) 
  called reduced_sum_of_pairs() with [[[[[0, 7], 4], [[7, 8], [6, 0]]], [8, 1]]]
  reduced_sum_of_pairs = [[[[0, 7], 4], [[7, 8], [6, 0]]], [8, 1]]
       expected result = 1384
      magnitude of sum = 1384
magnitudes5 operands(1) 
  called reduced_sum_of_pairs() with [[[[[1, 1], [2,

  reduced_sum_of_pairs = [[[[6, 6], [7, 6]], [[7, 7], [7, 0]]], [[[7, 7], [7, 7]], [[7, 8], [9, 9]]]]
       expected result = [[[[6, 6], [7, 6]], [[7, 7], [7, 0]]], [[[7, 7], [7, 7]], [[7, 8], [9, 9]]]]
      magnitude of sum = 4140
demo15 operands(2) 
  called reduced_sum_of_pairs() with [[[2, [[7, 7], 7]], [[5, 8], [[9, 3], [0, 2]]]], [[[0, [5, 8]], [[1, 7], [9, 6]]], [[4, [1, 2]], [[1, 4], 2]]]]
  reduced_sum_of_pairs = [[[[7, 8], [6, 6]], [[6, 0], [7, 7]]], [[[7, 8], [8, 8]], [[7, 9], [0, 6]]]]
       expected result = [[[[7, 8], [6, 6]], [[6, 0], [7, 7]]], [[[7, 8], [8, 8]], [[7, 9], [0, 6]]]]
      magnitude of sum = 3993
demo16 operands(2) 
  called reduced_sum_of_pairs() with [[[[0, [4, 5]], [0, 0]], [[[4, 5], [2, 6]], [9, 5]]], [7, [[[3, 7], [4, 3]], [[6, 3], [8, 8]]]]]
  reduced_sum_of_pairs = [[[[4, 0], [5, 4]], [[7, 7], [6, 0]]], [[8, [7, 7]], [[7, 9], [5, 0]]]]
       expected result = [[[[4, 0], [5, 4]], [[7, 7], [6, 0]]], [[8, [7, 7]], [[7, 9], [5, 0]]]]
      magnitude

In [17]:
part_2_notes = """

Again considering the last example homework assignment above:

[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]

The largest magnitude of the sum of any two snailfish 
numbers in this list is 3993. This is the magnitude of 

  [[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]] + 
  [[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]] =
  [[[[7,8],[6,6]],[[6,0],[7,7]]],[[[7,8],[8,8]],[[7,9],[0,6]]]]
  
"""

In [18]:
# part 2: calculate pairwise sums of all the operands
for source_name in ["puzzle"]: # data.keys():
    source = deepcopy(data[source_name])
    
    for i,operand_group in enumerate(source["operands"]):
        print(f"{source_name}{i}: operand_group = {operand_group}")
        
        maximum_magnitude = -infinity
        maximum_unreduced_sum = [0,0]
        for j,saved_operands0 in enumerate(operand_group):
            for k,saved_operands1 in enumerate(operand_group):
                if j != k:
                    operands0 = deepcopy(saved_operands0)
                    operands1 = deepcopy(saved_operands1)
                    
                    verbose(f"{source_name}{i}: operands{j} = {operands0} operands{k} = {operands1}")      
                    unreduced_sum = unreduced_sum_of_two_pairs(operands0,operands1)
                    verbose(f"         unreduced_sum = {unreduced_sum}")
                
                    while reduce( unreduced_sum ):
                        pass
                    reduced_sum = unreduced_sum
                    verbose(f"  reduced_sum_of_pairs = {reduced_sum}")
                
                    magnitude = pair_magnitude(reduced_sum)
                    verbose(f"      magnitude of sum = {magnitude}")
                
                    if magnitude > maximum_magnitude:
                        maximum_magnitude = magnitude
                        maximum_unreduced_sum = unreduced_sum
                    
        print(f"{source_name}{i}:")
        print(f"   maximum_magnitude = {maximum_magnitude}")
        print(f"   maximum_unreduced_sum = {maximum_unreduced_sum}\n{70 * '*'}")

puzzle0: operand_group = [[[[[2, 5], 4], [[1, 0], [8, 3]]], [[2, [2, 4]], [1, [3, 3]]]], [[[2, 2], [[4, 3], 3]], [[[8, 6], 3], [3, 7]]], [[[9, [4, 1]], [9, 0]], [6, [6, 0]]], [[[3, 9], [[4, 4], [2, 5]]], [[9, [8, 4]], 8]], [[[[0, 0], 9], [[9, 3], [8, 2]]], [2, [1, 3]]], [[[8, 4], 6], [[5, 1], [3, 6]]], [[[6, [7, 6]], [[2, 6], 5]], [[6, 4], 2]], [[1, [9, 7]], [[[5, 9], [9, 5]], [[7, 0], 1]]], [[[[5, 8], [9, 4]], [[9, 3], [7, 8]]], 8], [[[0, 9], [[6, 0], 7]], [[[7, 7], 6], [[9, 7], [0, 4]]]], [[[[4, 3], [9, 5]], [7, [7, 3]]], [[[2, 8], 9], 4]], [[7, 5], [8, 1]], [[4, 6], [[[0, 6], 6], [7, 4]]], [[[1, 8], [[1, 4], [1, 6]]], [3, 4]], [[[6, 5], [4, [7, 3]]], [[[0, 1], [8, 4]], [4, 8]]], [[5, 1], [9, [9, [3, 3]]]], [[[[7, 0], [2, 5]], 1], [9, [[2, 7], [4, 4]]]], [[[[5, 8], 8], 0], [8, [1, [2, 5]]]], [8, [[5, 4], 7]], [[[9, 8], [6, 7]], [[2, [2, 6]], [9, 6]]], [[[[2, 3], 7], 6], [[8, 6], 3]], [[[8, [7, 2]], 3], [[[3, 9], 4], [6, 8]]], [9, [[[6, 7], [6, 0]], [[3, 9], 8]]], [[[7, 7], [4, 7]], [